In [5]:
from langchain_community.chat_models import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_key=openai_api_key)

from discussion_agents.cog.agent.expel import ExpeLAgent

In [6]:
import joblib 

hotpotqa = joblib.load('../../../tests/assets/hotpotqa/hotpot-qa-distractor-sample.joblib')

In [7]:
agent = ExpeLAgent(
    llm=llm,
    self_reflect_llm=llm,
    action_llm=llm
)

In [1]:
from discussion_agents.cog.prompts.react import REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES

In [10]:
a == REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES + "\n(END OF EXAMPLES)\n"

False

In [24]:
a = """
Here are some examples:
{examples}
(END OF EXAMPLES)
"""

b = f"""
Here are some examples:
{REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES}
""" + \
"(END OF EXAMPLES)\n"

a.format(examples=REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES) == b

True

In [19]:
a

'\nHere are some examples:\n{examples}\n(END OF EXAMPLES)\n'

In [8]:
from discussion_agents.cog.agent.reflexion import ReflexionReActAgent

reflexion_agent = ReflexionReActAgent(
    self_reflect_llm=llm,
    action_llm=llm
)

In [9]:
reflexion_agent.generate(
    hotpotqa.iloc[0].question,
    hotpotqa.iloc[0].answer
)

PROMPT:

 
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the last passage successfully found by Search.
(3) Finish[answer], which returns the answer and finishes the task.
You have a maximum of 6 steps.

Here are some examples:

Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado

[(False,
  '',
  [('Thought: I need to search for VIVA Media AG and find out what their new acronym stands for after changing their name in 2004. ',
    'Action: Search[VIVA Media AG]',
    "Observation 1: Could not find [VIVA Media AG]. Similar: ['MTV Music (Polish TV channel)', 'Paramount International Networks', 'VIVA Plus', 'VIVA (German TV channel)', 'Viacom (1952–2005)', 'Vauxhall Viva', 'Sartorius AG', 'GfK Entertainment charts', 'Lindt', 'Spellbound Entertainment']"),
   ("Thought: I couldn't find VIVA Media AG, maybe I should try searching for VIVA Media AG name change 2004 to find their new acronym.",
    'Action: Search[VIVA Media AG name change 2004]',
    "Observation 2: Could not find [VIVA Media AG name change 2004]. Similar: ['Vauxhall Viva', 'GfK Entertainment charts', 'Opel Astra', 'Puma (brand)', 'Priscilla Presley', 'About You Now', 'Mediaspree', 'Daihatsu Mira', 'Sildenafil', 'Bosch (company)']"),
   ("Thought: I need to broaden my search and look for VIVA Media AG

: 

In [25]:
agent.insight_memory.insights

[]

In [26]:
agent.gather_experience(
    questions=hotpotqa.question.values[:1],
    keys=hotpotqa.answer.values[:1]
)

In [28]:
agent.insight_memory.insights

[{'insight': 'Break down complex tasks into smaller, manageable steps to improve efficiency and accuracy.',
  'score': 2},
 {'insight': 'Prioritize tasks based on urgency and importance to increase productivity and focus.',
  'score': 2}]

In [29]:
agent.experience_memory.experiences['trajectories'][0]

[(False,
  'Look up the history of VIVA Media AG on a different platform or source',
  [('Thought: I need to search VIVA Media AG to find out what their new acronym stands for after changing their name in 2004. ',
    'Action: Search[VIVA Media AG]',
    "Observation 1: Could not find [VIVA Media AG]. Similar: ['MTV Music (Polish TV channel)', 'Paramount International Networks', 'VIVA Plus', 'VIVA (German TV channel)', 'Viacom (1952–2005)', 'Vauxhall Viva', 'Sartorius AG', 'GfK Entertainment charts', 'Lindt', 'Spellbound Entertainment']"),
   ("Thought: I couldn't find VIVA Media AG, so I should search for VIVA Media AG's name change in 2004 to get the new acronym.",
    'Action: Search[VIVA Media AG name change 2004]',
    "Observation 2: Could not find [VIVA Media AG name change 2004]. Similar: ['Vauxhall Viva', 'GfK Entertainment charts', 'Opel Astra', 'Puma (brand)', 'Priscilla Presley', 'About You Now', 'Mediaspree', 'Daihatsu Mira', 'Sildenafil', 'Bosch (company)']"),
   ('Though

In [20]:
agent.experience_memory.success_traj_docs

[]

In [21]:
agent.experience_memory.vectorstore

In [22]:
agent.insight_memory.insights

[{'insight': 'Prioritize security and privacy when handling sensitive data.',
  'score': 2},
 {'insight': 'Regularly update and maintain the Docstore API environment for optimal performance.',
  'score': 2},
 {'insight': 'Regularly monitor and analyze system performance to identify and address any potential issues early on.',
  'score': 2}]